In [1]:
from sklearn.datasets import load_wine
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd

data = load_wine()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target)

scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns=X.columns)
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [2]:
class Perceptron:
    def __init__(self, learning_rate=0.01, max_iter=1000):
        self.learning_rate = learning_rate
        self.max_iter = max_iter
        self.weights = None
        self.bias = 0

    def fit(self, X_train, y_train):
        n_samples, n_features = X_train.shape
        self.weights = np.zeros(n_features)

        for _ in range(self.max_iter):
            for i in range(n_samples):
                xi = X_train.iloc[i]
                yi = y_train.iloc[i]
                linear_output = np.dot(xi, self.weights) + self.bias
                predicted_output = 1 if linear_output >= 0 else 0

                if predicted_output != yi:
                    self.weights += self.learning_rate * (yi - predicted_output) * xi
                    self.bias += self.learning_rate * (yi - predicted_output)

    def predict_proba(self, X):
        return np.dot(X, self.weights) + self.bias

    def predict(self, X):
        return np.where(self.predict_proba(X) >= 0, 1, 0)

In [3]:
 class OVRClassifier:
    def __init__(self, base_estimator_class, **kwargs):
        self.base_estimator_class = base_estimator_class
        self.kwargs = kwargs
        self.classifiers = {}

    def fit(self, X, y):
        self.classes_ = np.unique(y)

        for class_label in self.classes_:
            y_binary = (y == class_label).astype(int)
            classifier = self.base_estimator_class(**self.kwargs)
            classifier.fit(X, y_binary)

            self.classifiers[class_label] = classifier
    def predict(self, X):
        scores = {}
    
        for class_label, classifier in self.classifiers.items():
            if hasattr(classifier, "decision_function"):
                score = classifier.decision_function(X)
            elif hasattr(classifier, "predict_proba"):
                proba = classifier.predict_proba(X)
                if proba.ndim == 1:
                    score = proba
                else:
                    score = proba[:, 1]
            else:
                raise ValueError("Klasyfikator nie wspiera predict_proba ani decision_function")
    
            scores[class_label] = score
    
        all_scores = np.vstack([scores[class_label] for class_label in self.classes_]).T
        predicted_indices = np.argmax(all_scores, axis=1)
        return self.classes_[predicted_indices]

In [4]:
def confusion_matrix(y_true, y_pred, num_classes):
    cm = np.zeros((num_classes, num_classes), dtype=int)
    
    for true_label, pred_label in zip(y_true, y_pred):
        cm[true_label, pred_label] += 1
    return cm
    
def accuracy(y_true, y_pred):
    return np.sum(y_true == y_pred) / len(y_true)

def precision(y_true, y_pred, average='macro'):
    cm = confusion_matrix(y_true, y_pred, num_classes=len(np.unique(y_true)))
    
    if average == 'macro':
        precisions = []
        for i in range(cm.shape[0]):
            TP = cm[i, i]
            FP = np.sum(cm[:, i]) - TP
            precisions.append(TP / (TP + FP) if (TP + FP) != 0 else 0)
        return np.mean(precisions)
    
    elif average == 'micro':
        TP = np.sum(np.diagonal(cm))
        FP = np.sum(cm) - TP
        return TP / (TP + FP)

def recall(y_true, y_pred, average='macro'):
    cm = confusion_matrix(y_true, y_pred, num_classes=len(np.unique(y_true)))
    
    if average == 'macro':
        recalls = []
        for i in range(cm.shape[0]):
            TP = cm[i, i]
            FN = np.sum(cm[i, :]) - TP
            recalls.append(TP / (TP + FN) if (TP + FN) != 0 else 0)
        return np.mean(recalls)
    
    elif average == 'micro':
        TP = np.sum(np.diagonal(cm))
        FN = np.sum(cm) - TP
        return TP / (TP + FN)

def f1_score(y_true, y_pred, average='macro'):
    prec = precision(y_true, y_pred, average)
    rec = recall(y_true, y_pred, average)
    
    if average == 'macro':
        return 2 * (prec * rec) / (prec + rec) if (prec + rec) != 0 else 0
    
    elif average == 'micro':
        return 2 * (prec * rec) / (prec + rec) if (prec + rec) != 0 else 0


In [5]:
ovr_perceptron = OVRClassifier(Perceptron, learning_rate=0.001, max_iter=1000)
ovr_perceptron.fit(X_train, y_train)
y_pred_perc = ovr_perceptron.predict(X_test)

print("Perceptron (One-vs-Rest)")
cm_perc = confusion_matrix(y_test, y_pred_perc, num_classes=3)
print("Confusion matrix:\n", cm_perc)
print("Accuracy:", accuracy(y_test, y_pred_perc))
print("Precision (macro):", precision(y_test, y_pred_perc, average='macro'))
print("Recall (macro):", recall(y_test, y_pred_perc, average='macro'))
print("F1-score (macro):", f1_score(y_test, y_pred_perc, average='macro'))
print("Precision (micro):", precision(y_test, y_pred_perc, average='micro'))
print("Recall (micro):", recall(y_test, y_pred_perc, average='micro'))
print("F1-score (micro):", f1_score(y_test, y_pred_perc, average='micro'))

Perceptron (One-vs-Rest)
Confusion matrix:
 [[14  0  0]
 [ 0 14  0]
 [ 0  0  8]]
Accuracy: 1.0
Precision (macro): 1.0
Recall (macro): 1.0
F1-score (macro): 1.0
Precision (micro): 1.0
Recall (micro): 1.0
F1-score (micro): 1.0




In [6]:
from sklearn.svm import SVC

svc_ovr = OVRClassifier(SVC, kernel="linear", probability=True)
svc_ovr.fit(X_train, y_train)
y_pred_svc = svc_ovr.predict(X_test)

print("SVC (sklearn OneVsRestClassifier)")
cm_svc = confusion_matrix(y_test, y_pred_svc, num_classes=3)
print("Confusion matrix:\n", cm_svc)
print("Accuracy:", accuracy(y_test, y_pred_svc))
print("Precision (macro):", precision(y_test, y_pred_svc, average='macro'))
print("Recall (macro):", recall(y_test, y_pred_svc, average='macro'))
print("F1-score (macro):", f1_score(y_test, y_pred_svc, average='macro'))
print("Precision (micro):", precision(y_test, y_pred_svc, average='micro'))
print("Recall (micro):", recall(y_test, y_pred_svc, average='micro'))
print("F1-score (micro):", f1_score(y_test, y_pred_svc, average='micro'))

SVC (sklearn OneVsRestClassifier)
Confusion matrix:
 [[14  0  0]
 [ 0 14  0]
 [ 0  0  8]]
Accuracy: 1.0
Precision (macro): 1.0
Recall (macro): 1.0
F1-score (macro): 1.0
Precision (micro): 1.0
Recall (micro): 1.0
F1-score (micro): 1.0
